In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from langchain_core.documents import Document

laws_txt_splitter = RecursiveCharacterTextSplitter(chunk_size=800, 
                                               chunk_overlap=100,
                                               separators=['\n\n', '\n', ',', ';', '-', ':', ','])
with open('traffic_laws_alok_kumar_hyd_2016.txt', 'r', encoding='utf-8') as f:
    text=f.read()
chunks1=laws_txt_splitter.split_text(text)
df = pd.read_csv('Hyd_challan_fines - Sheet1.csv')
doc=[]
for i, row in df.iterrows():
    text=f'Offence : {row["Offences"]}\nSection : {row["Section"]}\nPenalty : {row["Maximum Penalty"]}'
    doc.append(Document(page_content=text, metadata={'row':i}))
csv_splitter_penalties = RecursiveCharacterTextSplitter(separators=['\n', '\n\n'], chunk_size=500, chunk_overlap=50)
chunks2 = csv_splitter_penalties.split_documents(doc)

In [12]:
print(chunks1[0])

G.S.R.634(E).— In exercise of the powers conferred by section 118 of the Motor Vehicles Act, 1988 (59 of
1988), and in supersession of the Rules of the Road Regulations, 1989, except as respects things done or omitted to be
done before such supersession, the Central Government hereby makes the following regulations for the driving of motor
vehicles, namely:- 
1. Short title, extent and commencement.
(1) These regulations may be called the Motor Vehicles (Driving)
Regulations, 2017.
(2) They shall come into force on the date of its publication in the Official Gazette.
2. Definitions.- (1) In these regulations, unless the context otherwise requires,-
(a) “Act” means the Motor Vehicles Act, 1988 (59 of 1988);


In [14]:
import numpy as np
embeddings1=model.encode(chunks1, convert_to_numpy=True)
chunks2_texts = [doc.page_content for doc in chunks2]
embeddings2=model.encode(chunks2_texts, convert_to_numpy=True)
embeddings=np.vstack([embeddings1, embeddings2])
chunks=chunks1+chunks2_texts

In [16]:
chunks2[0]

Document(metadata={'row': 0}, page_content='Offence : To drive a vehicle without license.\nSection : M.V.Act. 181\nPenalty : Rs. 500/- for two, three, four & Six wheelers.')

In [18]:
import faiss
import json

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
faiss.write_index(index, 'traffic_rules.index')

with open('chunks.json', 'w') as f:
    json.dump(chunks, f)